# Data Analysis

This notebook is devoted to the analysis of the DWAVE data and the comparison between them and the data generated by the Neural Network (MADE for the moment) 

In [4]:
import glob

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.utils.utils import plot_hist
from src.utils.montecarlo import neural_mcmc

## Check energy

Load dataset as it is saved by [DWAVE system](https://cloud.dwavesys.com/leap/login/?next=/leap/). Since there is a maximum time for the annealing process, data are saved in files with maximum size of 10k sample. In the same folder one can find also two energy file, one computed directly by DWAVE annealer and the other one with our custom algorithm. Both should give us the same result. So, here we load the dataset and we rearrange them in two files, namely train and validation dataset.

In [ ]:
files = glob.glob(f'configs_*')
#print(files)

arrs = []
for file in files:
    arrs.append(np.load(file))
dataset = np.concatenate(arrs, axis=0)
print(dataset.shape)
train_data, test_data = train_test_split(dataset, test_size=0.15)

# Comment off the following lines to save the datasets.
# np.save('DWAVE-train-484spins-1nn-100mu', train_data)
# np.save('DWAVE-test-484spins-1nn-100mu', test_data)
np.save("1000mu", dataset)


Here we load a small part of the original dataset in order to check if the energy is well computed.

In [ ]:
sample = np.load("../data/1-50k_open-simple_10mu/configs_0.npy")
dwave_eng = np.load("../data/1-50k_open-simple_10mu/dwave-engs_0.npy")
eng = np.loadtxt("../data/1-50k_open-simple_10mu/energies_0.txt")

print(f"Energy (from DWAVE) {dwave_eng[:4]}\nEnergy (our algo) {-eng[:4]*484}")

## Plot histograms

Here we want to check if the Neural Network has been well trained, a good measure could also be the mean energy of the DWAVE data and the generated data.

In [ ]:
path=["data/datasets/484-1nn-50mu/DWAVE-train-484spins-50mu.npy"]  #["/home/beppe/neural-mcmc/sample-100000_size-484_2021-11-15_14_31_46.npz"]
labels = [r"MADE+MCMC", r"Dwave 50$\mu$s",]# r"Dwave 100$\mu$s", r"NN Re-Weighted",]
truth = "data/seq_temp/484spins_open-1nn/single-beta0.8.npy" #"/home/beppe/neural-mcmc/data/datasets/484-1nn-100mu/DWAVE-train-484spins-1nn-100mu.npy"
couplings_path="data/couplings/484spins_open-1nn.txt"

engs, eng_truth = plot_hist(path, couplings_path, truth_path=truth, labels=labels, save=True)


## Results with Trained MADE on DWave Data

Computing the acceptance rate for some $\beta \in (0,5)$ we can notice that it exists an effective $\beta$, let's call it $\beta_{eff}$ that is increasing according to the annealing time selected in the D-Wave machine. 

In [5]:
datasets = ["data/datasets/484-1nn-10mu/DWAVE-test-484spins-10mu.npy", 
            "data/datasets/484-1nn-50mu/DWAVE-test-484spins-50mu.npy", 
            "data/datasets/484-1nn-100mu/DWAVE-test-484spins-1nn-100mu.npy"]
betas = np.arange(0.1, 3., step=0.1)

In [ ]:
acc_rates = []
for dataset in tqdm(datasets, leave=True):
    acc_rate = []
    for beta in betas:
        ar = neural_mcmc(beta, 100000, dataset, couplings_path, "made")
        acc_rate.append(ar)
    acc_rates.append(acc_rate)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8), facecolor='white')

plt.rcParams['mathtext.fontset']= "stix"
plt.rcParams['font.family']= 'STIXGeneral'

stringfont = 'serif'

plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', labelsize=10)

ax.set_xticklabels([r'0', r'0.5', r'1',r'1.5',r'2',r'2.5',r'3'], fontsize=12, fontfamily=stringfont)
ax.set_yticklabels([r'0',r'1',r'2',r'3',r'4',r'5'], fontsize=12, fontfamily=stringfont)

plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')

labels = [r"10 $\mu$s",r"50 $\mu$s"]

for i, acc_rate in enumerate(acc_rates):
    plt.plot(betas, acc_rate, "-.", label=labels[i])#, linewidth=3.)

plt.xlim(0,3)
plt.ylim(0,5)

plt.ylabel(r"$\mathrm{A_r}$[%]", fontsize=18, fontfamily=stringfont)
plt.xlabel(r"$\mathrm{\beta}$", fontsize=18, fontfamily=stringfont)

plt.legend(loc='best', fontsize=18, labelspacing=0.4, borderpad=0.2)

plt.savefig("ar-vs-beta.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')